In [1]:
from traci_backend import TraciBackend
import random
"""
We need to import some functions
"""

'\nWe need to import some functions\n'

In [2]:
tb = TraciBackend()


In [6]:
tb.start()

 Retrying in 1 seconds


In [4]:
tb.simulate_step()

In [5]:
tb.close()

In [53]:
class Agent():
    def __init__(self, discount, alpha, epsilon):
        self.discount = discount
        self.alpha = alpha
        self.epsilon = epsilon
        self.weights1 = [1] * 4 # for action1
        self.weights2 = [1] * 4 # for action2
        
    def getStateReward(self, edges):
        '''
        Fetch data from traci backend,
        return the state and reward of this step
        The reward is calculated using data from a specific step,
        not accumulated.
        '''
        state, reward, isEnd = [0] * 4, 0, True
        for n, edge in enumerate(edges):
            line_length, total_vehicle_num = 0, 0
            for i in range(3):
                lane = edge + '_' + str(i)
                line_length += tb.get_halt_vehicle_cnt(lane)
                total_vehicle_num += tb.get_vehicle_cnt(lane)
            if total_vehicle_num > 0 and isEnd:
                isEnd = False
            pass_vehicle_num = total_vehicle_num - line_length
#             pass_vehicle_num = total_vehicle_num
            state[(n&1) * 2] += line_length
            state[(n&1) * 2 + 1] += pass_vehicle_num
            reward += pass_vehicle_num - line_length
        print(state, reward, isEnd)
        return state, reward, isEnd
#         state = [time()] # add time stamp in SUMO
#         lengths = getAllQueueLength()
#         state += lengths # add four elements
#         passes = getLastStepPass()
#         state += passes # add four elements
#         return state, sum([passes[i] - lengths[i] for i in range(4)])
    
    def getQ(self, state):
        '''
        Use the function approximator, give out the approximated value
        '''
        q1, q2 = 0, 0
        for i in range(4):
            q1 += state[i] * self.weights1[i]
            q2 += state[i] * self.weights2[i]
        return q1, q2
        
    def eGreedy(self, q):
        '''
        Return the action selection 0 or 1 according to q-value
        '''
        q1, q2 = q[0], q[1]
#         print("The approximated q values =", q1, q2)
        rand = random.random()
        if q1 == q2:
            return 0 if rand < 0.5 else 1
        if rand < self.epsilon and q1 < q2 or rand > self.epsilon and q1 > q2:
#             print("We choose action1")
            return 0
        else:
#             print("We choose action2")
            return 1
        
        
    def updateWeight(self, curr_state, next_state, reward, action):
        '''
        Use approximate Q-learning, update weight in the approximator
        '''
        print("Update weights with current state", curr_state, "next state", next_state, "and reward", reward)
        q_curr = self.getQ(curr_state)[action] # current approximated q value
        q_next_act = self.getQ(next_state) # next approximated q value for both action
        next_act = self.eGreedy(q_next_act) # get the action by epsilon greedy
#         print("Epsilon greedy chooses the action", next_act + 1)
        q_next = q_next_act[next_act]
        delta = reward + self.discount * q_next - q_curr
        print("delta =", delta, "q_curr", q_curr, "q_next", q_next)
        # only update one weights array, either for action1 or action2
        if action:
            self.weights2 = [self.weights2[i] + self.alpha * delta * curr_state[i] for i in range(4)]
        else:
            self.weights1 = [self.weights1[i] + self.alpha * delta * curr_state[i] for i in range(4)]
#         print(self.weights1, self.weights2)
        return next_act, action ^ next_act
    
    def executeAction(self, action, change):
        '''
        Adjust the lights in sumo given the action 0 or 1
        '''
        if not change:
            return
        if change and action:
            tb.set_light_phase('0', 1)
        elif change and not action:
            tb.set_light_phase('0', 5)
        for _ in range(15):
            tb.simulate_step()
        
            
    
    def train(self, step_size):
        '''
        Train the agent with num_iter, observe the reward from traci backend every step_size period.
        step_size is the time before observing the state and reward
        '''
        tb = TraciBackend()
        tb.start()
        prev_state = [0] * 4
        next_act, change = 0, True
        weights_history = [[], []]
        isEnd = False
        while not isEnd:
#             state, reward = [0], 0
            self.executeAction(next_act, change)
            for _ in range(step_size):
                tb.simulate_step()
#             lanes = tb.get_lane_ids()
#             lights = tb.get_light_ids()
            state, reward, isEnd = self.getStateReward(['1si', '2si', '3si', '4si'])
            weights_history[0].append(self.weights1)
            weights_history[1].append(self.weights2)
            next_act, change = self.updateWeight(prev_state, state, reward, next_act)
            
#             for lane in lanes:
#                 line_length = tb.get_halt_vehicle_cnt(lane)
#                 total_vehicle_num = tb.get_vehicle_cnt(lane)
#                 pass_vehicle_num = total_vehicle_num - line_length
#                 state += [line_length, pass_vehicle_num]
#                 reward += pass_vehicle_num - line_length
            prev_state = state
            
#         print(weights_history)
        tb.close()
        return weights_history

In [54]:
if __name__ == '__main__':
    simulateAgent = Agent(0.5, 0.01, 0.05)
    for _ in range(10):
        simulateAgent.train(20)
        print(simulateAgent.weights1, simulateAgent.weights2)

 Retrying in 1 seconds
[0, 3, 0, 1] 4 False
Update weights with current state [0, 0, 0, 0] next state [0, 3, 0, 1] and reward 4
delta = 6.0 q_curr 0 q_next 4
[1, 4, 1, 2] 4 False
Update weights with current state [0, 3, 0, 1] next state [1, 4, 1, 2] and reward 4
delta = 4.4 q_curr 3.6 q_next 8
[5, 1, 0, 2] -2 False
Update weights with current state [1, 4, 1, 2] next state [5, 1, 0, 2] and reward -2
delta = -6.0 q_curr 8 q_next 8
[6, 3, 1, 3] -1 False
Update weights with current state [5, 1, 0, 2] next state [6, 3, 1, 3] and reward -1
delta = -1.891 q_curr 6.42 q_next 11.058
[0, 5, 6, 1] 0 False
Update weights with current state [6, 3, 1, 3] next state [0, 5, 6, 1] and reward 0
delta = -5.846 q_curr 11.058 q_next 10.424
[4, 8, 7, 2] -1 False
Update weights with current state [0, 5, 6, 1] next state [4, 8, 7, 2] and reward -1
delta = -2.0451200000000016 q_curr 7.978560000000002 q_next 13.86688
[10, 1, 2, 6] -5 False
Update weights with current state [4, 8, 7, 2] next state [10, 1, 2, 6] 

[2, 14, 9, 6] 9 False
Update weights with current state [16, 1, 14, 12] next state [2, 14, 9, 6] and reward 9
delta = 869.5313484349349 q_curr -1666.0323381488547 q_next -1611.0019794278396
[6, 9, 10, 9] 2 False
Update weights with current state [2, 14, 9, 6] next state [6, 9, 10, 9] and reward 2
delta = 133.6130571329968 q_curr 671.7547086961852 q_next 1606.735531658364
[12, 0, 5, 15] -2 False
Update weights with current state [6, 9, 10, 9] next state [12, 0, 5, 15] and reward -2
delta = -415.4802613932893 q_curr 1822.8507996075787 q_next 2818.7410764285787
[12, 4, 11, 7] -12 False
Update weights with current state [12, 0, 5, 15] next state [12, 4, 11, 7] and reward -12
delta = -867.3811924539515 q_curr 1469.0826970049675 q_next 1227.403009102032
[3, 15, 8, 11] 15 False
Update weights with current state [12, 4, 11, 7] next state [3, 15, 8, 11] and reward 15
delta = 449.2182910309689 q_curr -1532.1761168681844 q_next -2195.915651674431
[5, 12, 8, 3] 2 False
Update weights with current 

[8, 15, 3, 20] 24 False
Update weights with current state [4, 13, 10, 10] next state [8, 15, 3, 20] and reward 24
delta = 1154.8802206746345 q_curr -3756.8495367870078 q_next -5251.9386322247465
[8, 4, 21, 6] -19 False
Update weights with current state [8, 15, 3, 20] next state [8, 4, 21, 6] and reward -19
delta = 5721.985983219041 q_curr -5251.9386322247465 q_next 978.0947019885896
[4, 10, 6, 14] 14 False
Update weights with current state [8, 4, 21, 6] next state [4, 10, 6, 14] and reward 14
delta = 10759.502606223496 q_curr 978.0947019885896 q_next 23447.19461642417
[11, 4, 19, 11] -15 False
Update weights with current state [4, 10, 6, 14] next state [11, 4, 19, 11] and reward -15
delta = 7176.368072881283 q_curr 23447.19461642417 q_next 61277.12537861091
[3, 8, 13, 18] 10 False
Update weights with current state [11, 4, 19, 11] next state [3, 8, 13, 18] and reward 10
delta = -34772.7961694286 q_curr 61277.12537861091 q_next 52988.65841836462
[11, 6, 21, 7] -19 False
Update weights wi

[4, 3, 0, 0] -1 False
Update weights with current state [2, 7, 0, 0] next state [4, 3, 0, 0] and reward -1
delta = -5196643421.256883 q_curr 7553291495.035102 q_next 4713296149.556438
[5, 0, 0, 0] -5 False
Update weights with current state [4, 3, 0, 0] next state [5, 0, 0, 0] and reward -5
delta = -1937967720.1342802 q_curr 2734939942.4362984 q_next 1593944454.6040363
[5, 0, 0, 0] -5 False
Update weights with current state [5, 0, 0, 0] next state [5, 0, 0, 0] and reward -5
delta = -523478237.55838823 q_curr 1046956465.1167765 q_next 1046956465.1167765
[0, 0, 0, 0] 0 True
Update weights with current state [5, 0, 0, 0] next state [0, 0, 0, 0] and reward 0
delta = -811391259.2155018 q_curr 811391259.2155018 q_next 0.0
[105480863.69801524, 307634988.4377329, 45247016.774417154, 525152208.205979] [-4967430881.662813, -1406132043.7578142, -6154819081.871297, -4281761952.3102837]
 Retrying in 1 seconds
[0, 3, 0, 1] 4 False
Update weights with current state [0, 0, 0, 0] next state [0, 3, 0, 1]

[4, 17, 17, 9] 5 False
Update weights with current state [12, 6, 17, 10] next state [4, 17, 17, 9] and reward 5
delta = 3003095662338.367 q_curr -2849334580789.8135 q_next 307522163087.1078
[13, 4, 17, 13] -13 False
Update weights with current state [4, 17, 17, 9] next state [13, 4, 17, 13] and reward -13
delta = 7378057496796.711 q_curr 307522163087.1078 q_next 15371159319793.637
[8, 12, 13, 11] 2 False
Update weights with current state [13, 4, 17, 13] next state [8, 12, 13, 11] and reward 2
delta = 5620393311874.461 q_curr 15371159319793.637 q_next 41983105263332.195
[13, 5, 14, 9] -13 False
Update weights with current state [8, 12, 13, 11] next state [13, 5, 14, 9] and reward -13
delta = -20838520596877.23 q_curr 41983105263332.195 q_next 42289169332935.93
[5, 11, 5, 12] 13 False
Update weights with current state [13, 5, 14, 9] next state [5, 11, 5, 12] and reward 13
delta = -29013363752700.9 q_curr 42289169332935.93 q_next 26551611160444.062
[11, 9, 9, 4] -7 False
Update weights wi

...

[8, 16, 17, 12] 3 False
Update weights with current state [19, 6, 17, 11] next state [8, 16, 17, 12] and reward 3
delta = 2.6090045378254296e+56 q_curr 3.4207616768930896e+55 q_next 5.902161411029477e+56
[15, 5, 18, 12] -16 False
Update weights with current state [8, 16, 17, 12] next state [15, 5, 18, 12] and reward -16
delta = 4.06676090372219e+56 q_curr 5.902161411029477e+56 q_next 1.9937844629503335e+57
[8, 12, 13, 14] 5 False
Update weights with current state [15, 5, 18, 12] next state [8, 12, 13, 14] and reward 5
delta = -4.510197951113922e+56 q_curr 1.9937844629503335e+57 q_next 3.0855293356778826e+57
[17, 6, 18, 8] -21 False
Update weights with current state [8, 12, 13, 14] next state [17, 6, 18, 8] and reward -21
delta = -1.4672321687284664e+57 q_curr 3.0855293356778826e+57 q_next 3.236594333898832e+57
[22, 3, 25, 5] -39 False
Update weights with current state [17, 6, 18, 8] next state [22, 3, 25, 5] and reward -39
delta = 4.3797515602679914e+57 q_curr -5.215531314246756e+57 q_

[19, 5, 19, 9] -24 False
Update weights with current state [9, 13, 11, 21] next state [19, 5, 19, 9] and reward -24
delta = -9.012404485466012e+72 q_curr 1.5491078352309766e+73 q_next 1.2957347733687509e+73
[5, 16, 12, 12] 11 False
Update weights with current state [19, 5, 19, 9] next state [5, 16, 12, 12] and reward 11
delta = -9.089483418843771e+72 q_curr 1.2957347733687509e+73 q_next 7.735728629687476e+72
[13, 10, 15, 8] -10 False
Update weights with current state [5, 16, 12, 12] next state [13, 10, 15, 8] and reward -10
delta = 1.8535143830162363e+73 q_curr -3.948510450357294e+73 q_next -4.189992134682115e+73
[15, 3, 18, 10] -20 False
Update weights with current state [13, 10, 15, 8] next state [15, 3, 18, 10] and reward -20
delta = 5.915189923953662e+73 q_curr -4.189992134682115e+73 q_next 3.450395578543093e+73
[4, 22, 5, 10] 23 False
Update weights with current state [15, 3, 18, 10] next state [4, 22, 5, 10] and reward 23
delta = 7.324262584580518e+73 q_curr 3.450395578543093e+73

[14, 3, 9, 10] -10 False
Update weights with current state [10, 5, 6, 6] next state [14, 3, 9, 10] and reward -10
delta = -1.5912294391429452e+76 q_curr 5.717457443401432e+76 q_next 8.252456008516974e+76
[0, 5, 2, 8] 11 False
Update weights with current state [14, 3, 9, 10] next state [0, 5, 2, 8] and reward 11
delta = -3.065001252496859e+76 q_curr 3.1468032163707537e+76 q_next 1.6360392774778935e+75
[1, 14, 1, 11] 23 False
Update weights with current state [0, 5, 2, 8] next state [1, 14, 1, 11] and reward 23
delta = 3.369830687883863e+75 q_curr -3.31620788034844e+76 q_next -5.958449623120108e+76
[13, 3, 5, 8] -7 False
Update weights with current state [1, 14, 1, 11] next state [13, 3, 5, 8] and reward -7
delta = 1.3484295139266024e+76 q_curr -4.8234317507466793e+76 q_next -6.950004473640154e+76
[13, 0, 11, 4] -20 False
Update weights with current state [13, 3, 5, 8] next state [13, 0, 11, 4] and reward -20
delta = 2.3260017617680862e+76 q_curr -4.259328345664767e+76 q_next -3.86665316

[15, 8, 12, 7] -12 False
Update weights with current state [10, 12, 3, 18] next state [15, 8, 12, 7] and reward -12
delta = -2.1986226006464177e+77 q_curr 7.225543358250391e+77 q_next 1.0053841515207946e+78
[10, 9, 11, 7] -5 False
Update weights with current state [15, 8, 12, 7] next state [10, 9, 11, 7] and reward -5
delta = -5.272729700967182e+76 q_curr 7.80771530497658e+75 q_next -8.983916340939048e+76
[12, 6, 16, 5] -17 False
Update weights with current state [10, 9, 11, 7] next state [12, 6, 16, 5] and reward -17
delta = 2.3873363995476335e+77 q_curr -2.9334625401742887e+77 q_next -1.0922522812533104e+77
[21, 9, 20, 4] -28 False
Update weights with current state [12, 6, 16, 5] next state [21, 9, 20, 4] and reward -28
delta = 7.127284679940826e+77 q_curr -1.0922522812533104e+77 q_next 1.2070064797375032e+78
[9, 12, 8, 24] 19 False
Update weights with current state [21, 9, 20, 4] next state [9, 12, 8, 24] and reward 19
delta = 2.6081325187312723e+77 q_curr 1.2070064797375032e+78 q_n

KeyboardInterrupt: 

In [31]:
simulateAgent = Agent(0.5, 0.01, 0.05)

In [32]:
simulateAgent.train(10, 20)

 Retrying in 1 seconds
[0, 0, 1, 0, 1, 0, 2, 0, 0] 4
Update weights with current state [0, 0, 0, 0, 0, 0, 0, 0, 0] next state [0, 0, 1, 0, 1, 0, 2, 0, 0] and reward 4
The approximated q values = 4 4
Epsilon greedy chooses the action 1
delta = 6.0 q_curr 0 q_next 4
[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0] [1, 1, 1, 1, 1, 1, 1, 1, 1]
[0, 2, 0, 2, 5, 0, 2, 0, 0] 3
Update weights with current state [0, 0, 1, 0, 1, 0, 2, 0, 0] next state [0, 2, 0, 2, 5, 0, 2, 0, 0] and reward 3
The approximated q values = 11.0 11
Epsilon greedy chooses the action 1
delta = 4.5 q_curr 4.0 q_next 11.0
[1.0, 1.0, 1.045, 1.0, 1.045, 1.0, 1.09, 1.0, 1.0] [1, 1, 1, 1, 1, 1, 1, 1, 1]
[0, 0, 0, 9, 3, 0, 2, 0, 0] -4
Update weights with current state [0, 2, 0, 2, 5, 0, 2, 0, 0] next state [0, 0, 0, 9, 3, 0, 2, 0, 0] and reward -4
The approximated q values = 14.315 14
We choose action1
Epsilon greedy chooses the action 1
delta = -8.247499999999999 q_curr 11.405 q_next 14.315
[1.0, 0.8350500000000001, 1.045, 0.835

[[[1, 1, 1, 1, 1, 1, 1, 1, 1],
  [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0],
  [1.0, 1.0, 1.045, 1.0, 1.045, 1.0, 1.09, 1.0, 1.0],
  [1.0,
   0.8350500000000001,
   1.045,
   0.8350500000000001,
   0.632625,
   1.0,
   0.9250500000000001,
   1.0,
   1.0],
  [1.0,
   0.8350500000000001,
   1.045,
   -0.1336582500000001,
   0.30972225,
   1.0,
   0.7097815000000001,
   1.0,
   1.0],
  [1.0,
   0.8350500000000001,
   1.045,
   -0.1336582500000001,
   0.30972225,
   1.0,
   0.7097815000000001,
   1.0,
   1.0],
  [1.0,
   0.8350500000000001,
   1.045,
   -0.1336582500000001,
   0.30972225,
   1.0,
   0.7097815000000001,
   1.0,
   1.0],
  [1.0,
   0.8350500000000001,
   1.045,
   -0.1336582500000001,
   0.30972225,
   1.0,
   0.7097815000000001,
   1.0,
   1.0],
  [1.0,
   0.8350500000000001,
   0.7195884486249999,
   -0.1336582500000001,
   -0.01568930137500002,
   -0.7897635325625001,
   0.38436994862500007,
   -0.1389404298125001,
   0.8372942243125],
  [1.0,
   0.8350500000000001,
  